### SCRIVI FEATURES su .txt

Stampa feature di un DataFrame del tipo: <br>
indice  |  feature  |  tipo

In [ ]:
import os
import pandas as pd

# Prompt the user for the relative CSV file path
csv_path = "data/speed_dating.csv"

# Check if the file exists
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found: {csv_path}")

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_path, encoding="latin1")

# Compute maximum widths for index, feature name, and dtype strings
max_index_width   = len(str(len(df.columns) - 1))
max_feature_width = max(len(col) for col in df.columns)
max_dtype_width   = max(len(str(dtype)) for dtype in df.dtypes)

# Prepare header (optional)
header = (
    f"{'Idx'.ljust(max_index_width)}\t"
    f"{'Feature'.ljust(max_feature_width)}\t"
    f"{'Dtype'.ljust(max_dtype_width)}\n"
)
separator = "-" * (max_index_width + max_feature_width + max_dtype_width + 8) + "\n"

# Ensure the output directory exists
os.makedirs("data", exist_ok=True)

# Write the aligned feature types into a text file
output_file = "data/features_info_123.txt"
with open(output_file, "w") as f:
    f.write(header)
    f.write(separator)
    for i, (col, dtype) in enumerate(df.dtypes.items()):
        line = (
            f"{str(i).ljust(max_index_width)}\t"
            f"{col.ljust(max_feature_width)}\t"
            f"{str(dtype).ljust(max_dtype_width)}\n"
        )
        f.write(line)

print(f"Feature types have been saved to '{output_file}'")

Feature types have been saved to 'data/features_info_195.txt'


Stampa label del DataFrame - solo label

In [5]:
import os
import pandas as pd

csv_path = "data/speed_dating.csv"
output_file = "data/features_labels_123.txt"
df = pd.read_csv(csv_path, encoding="latin1")

# write on the txt file all the attribute features
with open(output_file, "w") as f:
    for col in df.columns:
        f.write(f"{col}\n")

### Genera features_to_keep.txt

In [ ]:
import os
import pandas as pd

# Richiedi il percorso relativo del file CSV
csv_path = "data/speed_dating.csv"

# Verifica che il file esista
if not os.path.exists(csv_path):
    raise FileNotFoundError(f"File non trovato: {csv_path}")

# Carica il CSV in un DataFrame
df = pd.read_csv(csv_path)

# Calcola la larghezza massima dei nomi delle feature per un allineamento uniforme
max_width = max(len(col) for col in df.columns)

# Assicurati che la cartella di output esista
os.makedirs("data", exist_ok=True)

# Scrivi il file di testo con le feature e il flag 'y'
output_file = "data/features_to_keep.txt"
with open(output_file, "w", encoding="utf-8") as f:
    for i, col in enumerate(df.columns):
        # Utilizza ljust() per allineare il nome della feature a sinistra con una larghezza fissa
        # e separa con un tab la flag 'y'
        f.write(f"{str(i).rjust(3)}\t{col.ljust(max_width)}\t y\n")

print(f"Il file di selezione feature è stato salvato in: '{output_file}'")

# Check di consistenza

Consistenza 'race' == 'race_o' → "samerace"

In [ ]:
# conta quante righe hanno attributo 'race' e 'race_o' con valori non nulli
# dopodichè controlla (fra queste righe) se 'race' == 'race_o' e se la feature "samerace" è ben settata

import pandas as pd

# 1. Filtra le righe in cui 'race' e 'race_o' non sono nulli
non_null_rows = df[df['race'].notna() & df['race_o'].notna()]
print(f"Numero di righe con 'race' e 'race_o' non nulli: {len(non_null_rows)}")

# 2. Tra queste righe, conta quante hanno 'race' == 'race_o'
same_race_rows = non_null_rows[non_null_rows['race'] == non_null_rows['race_o']]
print(f"Numero di righe in cui 'race' == 'race_o': {len(same_race_rows)}")

# 3. Verifica se 'samerace' è coerente
# Assumiamo che:
#    - se 'race' == 'race_o' allora 'samerace' debba essere 1
#    - se 'race' != 'race_o' allora 'samerace' debba essere 0
# Consideriamo solo le righe in cui 'samerace' non è nullo
valid_rows = non_null_rows[non_null_rows['samerace'].notna()]

# Rileva le righe in cui 'samerace' non è impostato correttamente
inconsistent = valid_rows[
    ((valid_rows['race'] == valid_rows['race_o']) & (valid_rows['samerace'] != 1)) |
    ((valid_rows['race'] != valid_rows['race_o']) & (valid_rows['samerace'] != 0))
]
print(f"Numero di righe con 'samerace' non coerente: {len(inconsistent)}")

if len(inconsistent) == 0:
    print("La feature 'samerace' è ben settata.")
else:
    print("La feature 'samerace' presenta delle incongruenze.")

Consistenza 'decision' == 'decision_o' → "match"

In [ ]:
import pandas as pd

# Consider only rows where 'decision', 'decision_o' and 'match' are not null
valid_rows = df[df['decision'].notna() & df['decision_o'].notna() & df['match'].notna()]

# Define the condition:
# - If both 'decision' and 'decision_o' are 1, then 'match' must be 1.
# - In all other cases, 'match' must be 0.
inconsistent_rows = valid_rows[
    ((valid_rows['decision'] == 1) & (valid_rows['decision_o'] == 1) & (valid_rows['match'] != 1)) |
    (~((valid_rows['decision'] == 1) & (valid_rows['decision_o'] == 1)) & (valid_rows['match'] != 0))
]

print(f"Total valid rows: {len(valid_rows)}")
print(f"Number of inconsistent rows: {len(inconsistent_rows)}")
if len(inconsistent_rows) == 0:
    print("The 'match' column is correctly set based on 'decision' and 'decision_o'.")
else:
    print("Inconsistencies found in the 'match' column. Review these rows:")
    print(inconsistent_rows[['decision', 'decision_o', 'match']])

Consistenza discretizzazione di 'interests_correlate' in 'd_interests_correlate'

In [ ]:
import pandas as pd

# Definisci i bin e le etichette per la discretizzazione
bins = [-1, 0, 0.33, 1]
labels = ['[-1-0]', '[0-0.33]', '[0.33-1]']

# Discretizza la colonna 'interests_correlate'
# include_lowest=True assicura che il valore minimo (-1) venga incluso nel primo bin
df['d_interests_correlate_check'] = pd.cut(df['interests_correlate'], bins=bins, labels=labels, include_lowest=True)

# Calcola i conteggi per ogni categoria (includendo anche i NaN)
discretized_counts = df['d_interests_correlate_check'].value_counts(dropna=False).sort_index()
nan_count = df['interests_correlate'].isna().sum()

print("Conteggi ottenuti dalla discretizzazione:")
print(discretized_counts)
print(f"\nNumero di NaN in 'interests_correlate': {nan_count}")

# --- Confronto con i valori attesi ---
# Valori attesi:
# [-1-0]      2384
# [0-0.33]    3063
# [0.33-1]    2931
# NaN         158

expected_counts = {
    '[-1-0]': 2384,
    '[0-0.33]': 3063,
    '[0.33-1]': 2931,
    'NaN': 158
}

# Estrai i conteggi per le categorie discretizzate
obtained = {
    '[-1-0]': int(discretized_counts.get('[-1-0]', 0)),
    '[0-0.33]': int(discretized_counts.get('[0-0.33]', 0)),
    '[0.33-1]': int(discretized_counts.get('[0-0.33]', 0)) if '[0-0.33]' not in discretized_counts else int(discretized_counts.get('[0.33-1]', 0))
}

# Nota: Per gestire i NaN, usiamo:
obtained_nan = nan_count

print("\nConfronto con i valori attesi:")
for key in ['[-1-0]', '[0-0.33]', '[0-0.33]', '[0-0.33]']:
    # Evitiamo duplicati se ci fossero
    pass

# Più semplicemente, stampiamo il confronto riga per riga:
print(f"[-1-0]: ottenuto {int(discretized_counts.get('[-1-0]', 0))}, atteso {expected_counts['[-1-0]']}")
print(f"[0-0.33]: ottenuto {int(discretized_counts.get('[0-0.33]', 0))}, atteso {expected_counts['[0-0.33]']}")
print(f"[0.33-1]: ottenuto {int(discretized_counts.get('[0.33-1]', 0))}, atteso {expected_counts['[0.33-1]']}")
print(f"NaN:       ottenuto {obtained_nan}, atteso {expected_counts['NaN']}")

### Arrotondamento/Clipping per post imputer

In [ ]:
# --- 5. Verifica e Pulizia Post-Imputazione (Arrotondamento/Clipping) ---
print("--- Arrotondamento/Clipping Post-Imputazione ---")
print("Valori unici PRIMA dell'arrotondamento:")
for col in cols_to_encode:
     if col in df_imputed_RF.columns:
         # Usa try-except perché .unique() potrebbe fallire se la colonna non esiste
         try:
            print(f"{col}: {df_imputed_RF[col].unique()}")
         except KeyError:
             pass # La colonna non era nel df imputato

# Arrotonda e forza nel range valido le colonne originariamente categoriche
for col in cols_to_encode:
     if col in df_imputed_RF.columns:
        original_map = encoding_maps.get(col) # Prendi la mappa originale
        if original_map: # Assicurati che esista una mappa per questa colonna
            min_val = min(original_map.values())
            max_val = max(original_map.values())

            # Controlla se la colonna è float (potrebbe essere int se non c'erano NaN originali)
            if pd.api.types.is_float_dtype(df_imputed_RF[col]):
                df_imputed_RF[col] = df_imputed_RF[col].round() # Arrotonda

            # Converti in intero e fai il clipping in ogni caso per sicurezza
            df_imputed_RF[col] = df_imputed_RF[col].astype(int).clip(min_val, max_val)


print("\nValori unici DOPO arrotondamento/clipping:")
for col in cols_to_encode:
     if col in df_imputed_RF.columns:
         try:
             print(f"{col}: {df_imputed_RF[col].unique()}")
         except KeyError:
             pass

### Check su sole righe con NaN, data una precisa feature

In [ ]:
attribute_to_check= 'age'
nan_indices = df[df[attribute_to_check].isna()].index

In [ ]:
print(df.age_o.dtype)
print("----------RF----------------")
df_imputed_RF.loc[nan_indices, attribute_to_check].value_counts().sort_index()

In [ ]:
print("----------def----------------")
df_imputed_def.loc[nan_indices, attribute_to_check].value_counts().sort_index()

### Stampa grafico di quanti NaN nelle righe

In [ ]:
# Calcola il numero di NaN per ogni riga
nan_counts = df.isna().sum(axis=1)

# Calcola la distribuzione delle frequenze e ordina per indice
freq = nan_counts.value_counts().sort_index()

# Crea il bar plot
plt.figure(figsize=(8, 4))
ax = sns.barplot(x=freq.index, y=freq.values, palette='viridis')

# Etichette e titolo
plt.xlabel("Number of Missing Values per Row")
plt.ylabel("Frequency")
plt.title("Distribution of Missing Values per Row")

# Aggiungi le annotazioni con il valore assoluto sopra ogni barra
for p in ax.patches:
    ax.annotate(f'{int(p.get_height())}',
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha='center', va='center',
                xytext=(0, 5), textcoords='offset points')

plt.show()

### One Hot Encoding

In [ ]:
categorical_features = ['gender', 'race', 'race_o', 'field', 'samerace']
features_to_OHEncode = ['race', 'race_o', 'field']

# converti le colonne il cui dtype è float o category in int
for col in features_to_OHEncode:
    if col in df_imputed_MF.columns and df_imputed_MF[col].dtype in [np.float64, 'category']:
        df_imputed_MF[col] = df_imputed_MF[col].round().astype(int)
        print(f"Colonna '{col}' convertita in int.")

# --- 2. Applicazione del One-Hot Encoding con drop='first' ---
# pd.get_dummies è la funzione da usare.
# - df_imputed_MF: il DataFrame da trasformare.
# - columns: la lista delle colonne su cui applicare la codifica.
# - drop_first=True: elimina la prima categoria per ogni feature per evitare la multicollinearità.
# - dtype=int: opzionale, per avere 0 e 1 come interi invece che float.
df_OHEncoded = pd.get_dummies(
    df_imputed_MF,
    columns=features_to_OHEncode,
    drop_first=True,
    dtype=int
)

print("\n--- DataFrame dopo One-Hot Encoding (df_OHEncoded) ---")
print(df_OHEncoded.head()) # Mostra le prime righe del nuovo DataFrame
print(f"\nNuove dimensioni: {df_OHEncoded.shape}")
print(f"\nNuove colonne: {df_OHEncoded.columns.tolist()}")

### Dtype categorical features

In [ ]:
categorical_features = ['gender', 'race', 'race_o', 'field', 'samerace']

for col in categorical_features:
    if col in df.columns:
        print(f"\nColonna: {col}")
        print(f"Tipo: {df[col].dtype}")
        print(df[col].value_counts(dropna=False).sort_index())
    else:
        print(f"Colonna '{col}' non trovata nel DataFrame.")